
# Polynomial Regression (No Pipelines)

This notebook demonstrates **polynomial regression** in scikit-learn **without** using `Pipeline`.
We'll explicitly expand features with `PolynomialFeatures`, fit `LinearRegression`/`Ridge`, plot the fitted curves, and compare metrics.



## Learning goals

- Expand features with `PolynomialFeatures.fit_transform` manually.
- Fit `LinearRegression` and `Ridge` on the expanded arrays.
- Compare different polynomial degrees and discuss under/overfitting.
- Evaluate with MSE and \(R^2\) on train/test sets.


In [ ]:

# Setup: imports
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error, r2_score

np.set_printoptions(precision=3, suppress=True)



## 1) Create a curved dataset

We'll synthesize data from a quadratic curve with noise:
\[ y = 0.7 x^2 - 2 x + 5 + \epsilon, \quad \epsilon \sim \mathcal{N}(0, 3) \]


In [ ]:

# Reproducibility
rng = np.random.default_rng(123)

# Features (x) and target (y)
X = rng.uniform(-4, 4, 120)              # x-values between -4 and 4
noise = rng.normal(0, 3, size=120)       # Gaussian noise
y = 0.7 * X**2 - 2 * X + 5 + noise       # Quadratic relationship + noise

# scikit-learn expects 2D X
X = X.reshape(-1, 1)

print("X shape:", X.shape, " y shape:", y.shape)



### Quick look at the data


In [ ]:

plt.figure()
plt.scatter(X, y)
plt.title("Synthetic data: curved with noise")
plt.xlabel("x")
plt.ylabel("y")
plt.show()



## 2) Split the data into train and test sets


In [ ]:

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)
print("Train size:", X_train.shape[0], " Test size:", X_test.shape[0])



## 3) Fit linear vs. polynomial models (no pipelines)

We'll compare a plain linear model (degree=1) with polynomial degrees 2 and 5.
We **manually** call `PolynomialFeatures.fit_transform(X_train)` and `.transform(X_test)`.


In [ ]:

def fit_and_score_degree(degree=2, use_ridge=False, alpha=1.0):
    # Create polynomial feature transformer (no pipelines)
    poly = PolynomialFeatures(degree=degree, include_bias=False)
    Xtr_poly = poly.fit_transform(X_train)
    Xte_poly = poly.transform(X_test)

    # Choose estimator
    if use_ridge:
        reg = Ridge(alpha=alpha, random_state=42)
    else:
        reg = LinearRegression()

    reg.fit(Xtr_poly, y_train)

    # Predictions
    yhat_tr = reg.predict(Xtr_poly)
    yhat_te = reg.predict(Xte_poly)

    # Metrics
    mse_tr = mean_squared_error(y_train, yhat_tr)
    mse_te = mean_squared_error(y_test, yhat_te)
    r2_tr = r2_score(y_train, yhat_tr)
    r2_te = r2_score(y_test, yhat_te)

    return {
        "poly": poly, "reg": reg,
        "mse_tr": mse_tr, "mse_te": mse_te,
        "r2_tr": r2_tr, "r2_te": r2_te
    }

results = {
    "Linear (deg=1)": fit_and_score_degree(degree=1),
    "Poly deg=2": fit_and_score_degree(degree=2),
    "Poly deg=5": fit_and_score_degree(degree=5),
}

for name, res in results.items():
    coef_shape = res["reg"].coef_.shape
    print(name)
    print("  Coef dim:", coef_shape)
    print(f"  MSE train={res['mse_tr']:.3f} | test={res['mse_te']:.3f}")
    print(f"  R^2  train={res['r2_tr']:.3f} | test={res['r2_te']:.3f}")



### Visualize fits

We'll plot the training points and each model's predicted curve.


In [ ]:

# Smooth x-grid for drawing curves
x_line = np.linspace(X.min(), X.max(), 400).reshape(-1, 1)

plt.figure()
plt.scatter(X_train, y_train, label="train")
plt.scatter(X_test, y_test, label="test")

for name, res in results.items():
    poly, reg = res["poly"], res["reg"]
    x_line_poly = poly.transform(x_line)
    y_line = reg.predict(x_line_poly)
    plt.plot(x_line, y_line, label=name)

plt.title("Polynomial Regression fits by degree (no pipelines)")
plt.xlabel("x")
plt.ylabel("y")
plt.legend()
plt.show()



## 4) Overfitting and regularization (Ridge, no pipelines)

High-degree polynomials may **overfit**: very low training error but poor test performance.
We'll try `Ridge` (L2 regularization) with various `alpha` values.


In [ ]:

deg = 10
alpha_values = [0.0, 1.0, 10.0, 100.0]  # 0.0 ~ LinearRegression

def fit_degree_with_alpha(deg, alpha):
    poly = PolynomialFeatures(degree=deg, include_bias=False)
    Xtr_poly = poly.fit_transform(X_train)
    Xte_poly = poly.transform(X_test)

    if alpha == 0.0:
        reg = LinearRegression()
    else:
        reg = Ridge(alpha=alpha, random_state=42)

    reg.fit(Xtr_poly, y_train)
    tr_pred = reg.predict(Xtr_poly)
    te_pred = reg.predict(Xte_poly)

    return poly, reg, tr_pred, te_pred

print(f"Degree={deg} with different Ridge alpha values")
for alpha in alpha_values:
    poly, reg, tr_pred, te_pred = fit_degree_with_alpha(deg, alpha)
    print(f"alpha={alpha:>5} | MSE train={mean_squared_error(y_train, tr_pred):.3f} | test={mean_squared_error(y_test, te_pred):.3f} | R^2 test={r2_score(y_test, te_pred):.3f}")


In [ ]:

# Visual: degree-10 curves with different alpha (no pipelines)
plt.figure()
plt.scatter(X_train, y_train, label="train")
plt.scatter(X_test, y_test, label="test")

x_line = np.linspace(X.min(), X.max(), 400).reshape(-1, 1)

for alpha in [0.0, 10.0, 100.0]:
    poly = PolynomialFeatures(degree=10, include_bias=False)
    poly.fit(X_train)
    x_line_poly = poly.transform(x_line)

    if alpha == 0.0:
        reg = LinearRegression()
        name = "deg=10 (no ridge)"
    else:
        reg = Ridge(alpha=alpha, random_state=42)
        name = f"deg=10 + Ridge (alpha={alpha})"

    Xtr_poly = poly.transform(X_train)
    reg.fit(Xtr_poly, y_train)
    y_line = reg.predict(x_line_poly)
    plt.plot(x_line, y_line, label=name)

plt.title("Effect of Ridge regularization at high degree (no pipelines)")
plt.xlabel("x")
plt.ylabel("y")
plt.legend()
plt.show()



## 5) Inspect learned coefficients (degree 2)

Let's print the learned coefficients for degree 2 to see how the model captures the quadratic shape.


In [ ]:

# Refit a clean degree-2 model to show coefficients
poly2 = PolynomialFeatures(degree=2, include_bias=False)
Xtr2 = poly2.fit_transform(X_train)
reg2 = LinearRegression().fit(Xtr2, y_train)

feature_names = ["x", "x^2"]
for name, val in zip(feature_names, reg2.coef_.ravel()):
    print(f"{name:<4} -> {val: .4f}")
print("Intercept ->", reg2.intercept_)



## ✍️ Mini-exercises

1. Change the polynomial **degree** in the comparison section (e.g., try 3 and 8). How do MSE and \(R^2\) move?
2. Increase the **noise** scale in the data. What happens to the chosen degree's performance?
3. Compare `LinearRegression` vs `Ridge` across degrees (e.g., 2–10). Which `alpha` generalizes best?
4. Replace the synthetic data with a **real dataset** (e.g., one feature from a CSV) and repeat the analysis.
5. Set `include_bias=True` in `PolynomialFeatures` and observe how the intercept changes.



## Summary

- Polynomial regression = linear regression on expanded features \([x, x^2, x^3, \ldots]\)).
- Here we **did not** use pipelines; we called `fit_transform`/`transform` explicitly.
- Higher degree increases flexibility but can overfit; regularization (e.g., Ridge) helps.
- Always evaluate on held-out data (MSE, \(R^2\)).
